---
Stellen Sie bitte sicher, dass alles wie vorhergesehen läuft, bevor Sie dieses Übungsblatt abgeben. **Starten Sie den Kernel neu** (in der Menüleiste die Option Kernel$\rightarrow$Restart auswählen) und **validieren** Sie anschließend das Übungsblatt (in der Menüleiste auf Validate klicken) um Rückmeldung zu eventuellen fehlenden oder fehlerhaften Eingaben zu erhalten. 

Füllen Sie alle Stellen im Übungsblatt aus, welche entweder `DEIN CODE HIER` oder "DEINE ANTWORT HIER" enthalten. Geben Sie unterhalb Ihren vollständigen Namen an.

Wenn Sie Code-Bestandteile aus anderen Quellen (wie z.B. Stackoverflow) kopieren, dann machen sie den kopierten Code in ihrer Quellcodedatei kenntlich und fügen eine Referenz auf die Quelle als Kommentar hinzu.

Wenn Sie die Aufgaben in einer Gruppe erledigen, dann fügen Sie die Namen aller Gruppenmitglieder in der nachfolgende Zelle zu `Name` und zusätzlich als Kommentar am Anfang Ihrer Quellcodedatei hinzu.

---

In [ ]:
NAME = "Gernot Luidolt"

---

<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px;width: 180px"><img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo"></td>
        <td style="width: 100%">
            <div style="text-align:right; width: 100%; text-align:right"><font style="font-size:38px"><b>Methoden der Softwareentwicklung I</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>WS 2023/24</b></font></div>
        </td>
    </tr>
</table>

---
# Übungsblatt 1

In [ ]:
import jagl
import os

---
Abgabe bis <b>Sonntag, 3. Dezember 2023, 23:55 Uhr</b>

---
### Achtung:

*Bitte speichern Sie Ihre Quellcode-Dateien (&ast;.c) im selben Ordner wie dieses Übungsblatt. Beachten Sie außerdem die Kommentare zur richtigen Benennung Ihrer Dateien. Nur so ist eine korrekte Abgabe und Auswertung Ihrer Lösungen möglich.*

---

## 1.1 Summenberechnung (25%)

Implementieren Sie eine Funktion, welche bei der Zahl 2 startet und jede achte Zahl, die noch kleiner als 1000 ist, zu der bisherigen Summe (d.h. 2 + 10 + 18 + 26 + ...) hinzuaddiert. Am Ende soll ihr Programm die Summe dieser Berechnung ausgeben.

Speichern Sie Ihre Lösung in der Datei `sum_loop.c` ab und laden Sie diese auf JupyterHub in denselben Ordner wie den Übungszettel hoch.

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 1")

# delete all build artifacts
jagl.remove_paths(files=["sum_loop", "*.o"])

In [4]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "sum_loop" sein sollte.
# z.B. gcc -Wall -Werror -o sum_loop mein_quellcode.c

gcc -Wall -Werror -o sum_loop sum_loop.c

In [5]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_1_1_solved = True

In [6]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./sum_loop

62250


In [ ]:
executable_name = "sum_loop"
source_files = ["sum_loop.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_1_1(result, suite, case):
    varname = "exercise_1_1_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        result.setSucceeded("Exercise solved.")
    else:
        result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_1_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_1_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, [], None, timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")

---

## 1.2  Bestimmung von ungeraden Zahlen (25%)

Implementieren Sie ein Programm, welche für jede beliebige Integer-Zahl bestimmen kann, ob diese ungerade ist. Die zu untersuchende Zahl soll mittels Kommandozeilenparameter an das Programm übergeben werden. Falls eine Zahl ungerade ist, soll die Zeichenkette "true" ausgegeben werden. Falls es sich um eine gerade Zahl handelt, dann die Zeichenkette "false".

Speichern Sie Ihre Lösung in der Datei `is_odd.c` ab und laden Sie diese auf JupyterHub in denselben Ordner wie den Übungszettel hoch.

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 2")

# delete all build artifacts
jagl.remove_paths(files=["is_odd", "*.o"])

In [2]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "is_odd" sein sollte.
# z.B. gcc -Wall -Werror -o is_odd mein_quellcode.c

gcc -Wall -Werror -o is_odd is_odd.c

In [3]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_1_2_solved = True

In [4]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./is_odd 13

true


In [5]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./is_odd 42

false


In [ ]:
executable_name = "is_odd"
source_files = ["is_odd.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_2_1(result, suite, case):
    varname = "exercise_1_2_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        result.setSucceeded("Exercise solved.")
    else:
        result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_2_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_2_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, [], None, timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")

---

## 1.3 Pyramide (25%)

Implementieren Sie ein Programm, welches exakt folgende Ausgabe erzeugt:
```
  *  
 ***
*****
```

Dabei soll mindestens eine `for`-Schleife für die Ausgabe verwendet werden.

Speichern Sie Ihre Lösung in der Datei `pyramid.c` ab und laden Sie diese auf JupyterHub in denselben Ordner wie den Übungszettel hoch.

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 3")

# delete all build artifacts
jagl.remove_paths(files=["pyramid", "*.o"])

In [9]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name ihrer Quellcode-Datei "pyramid.c" sein sollte.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "pyramid" sein sollte.
# z.B. gcc -Wall -Werror -o pyramid pyramid.c

gcc -Wall -Werror -o pyramid pyramid.c

In [10]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_1_3_solved = True

In [11]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./pyramid

  *  
 ***
*****


In [ ]:
executable_name = "pyramid"
source_files = ["pyramid.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_3_1(result, suite, case):
    varname = "exercise_1_3_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        result.setSucceeded("Exercise solved.")
    else:
        result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_3_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_1_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, [], None, timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")

---

## 1.4 Zweierpotenzen (25%)

Implementieren Sie ein Programm, welches für einen beliebigen positiven Integer ermitteln kann, ob dieser eine Zweierpotenz (<MATH><var>2<sup>0</sup></var></MATH>, <MATH><var>2<sup>1</sup></var></MATH>, <MATH><var>2<sup>2</sup></var></MATH>, <MATH><var>2<sup>3</sup></var></MATH>,...) ist. Der zu untersuchende Integer soll dabei als Kommandozeilenparameter an das Programm übergeben werden. Falls es sich bei dem Integer um eine Zweierpotenz handelt, dann soll der entsprechende Exponent ausgegeben werden (z.B. für 32 soll 5 ausgegeben werden, weil <MATH><var>2<sup>5</sup></var>= <var>32</var></MATH>). Handelt es sich um keine Zweierpotenz, dann soll keinerlei Ausgabe erfolgen.

Speichern Sie Ihre Lösung in der Datei `powerof2.c` ab und laden Sie diese auf JupyterHub in denselben Ordner wie den Übungszettel hoch.

In [ ]:
# new test suite
jagl.testsuite_begin("Exercise 4")

# delete all build artifacts
jagl.remove_paths(files=["powerof2", "*.o"])

In [12]:
%%bash

# Schreiben Sie in diese Zelle den Befehl der dafür nötig ist um Ihr Programm zu kompilieren.
# Beachten Sie dabei, dass der Name Ihres ausführbaren Programms "powerof2" sein sollte.
# z.B. gcc -Wall -Werror -o powerof2 mein_quellcode.c


gcc -Wall -Werror -o powerof2 powerof2.c

In [13]:
# Weisen Sie der untenstehenden Variable den Wert True zu, sobald Sie die Aufgabe erfolgreich erledigt haben!

# Datentyp: bool
exercise_1_4_solved = True

In [14]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./powerof2 32

5


In [15]:
%%bash

# Benutzen Sie diese Zelle um Ihr Ergebnis zu testen.
# Damit diese Zelle ausgeführt werden kann, müssen Sie zuerst Ihr Programm erfolgreich kompilieren.

./powerof2 7

In [ ]:
executable_name = "powerof2"
source_files = ["powerof2.c"]
cpp_flags = []

In [ ]:
@jagl.testcase("1", desc="Exercise Solved")
def testcase_4_1(result, suite, case):
    varname = "exercise_1_4_solved"
    result.setSucceeded()
    jagl.check_variable_exists_and_has_type(result, globals(), varname, bool)
    if result.isSucceeded():
        result.setSucceeded("Exercise solved.")
    else:
        result.setFailed("Exercise not solved.")

In [ ]:
@jagl.testcase("2", desc="Compiling program", deps=["1"])
def testcase_4_2(result, suite, case):
    if os.path.isfile(executable_name):
        result.setSucceeded("The program could be compiled successfully.")
        
    else:
        res = jagl.exec_bin("gcc", ["-o", executable_name] + source_files + cpp_flags)
        if res.state == jagl.ExecutionResult.SUCCESS:
            stdout = jagl.bytes_decode(res.stdout).strip()
            stderr = jagl.bytes_decode(res.stderr).strip()
            if res.exitCode == 0:
                if os.path.isfile(executable_name):
                    result.setSucceeded("The program could be compiled successfully.")
                else:
                    result.setFailed("gcc was executed successfully, but I couldn't find an executable.")
                if len(stderr) > 0:
                    result.setSucceeded(["However, there were warnings."] + stderr.split("\n"))
            else:
                result.setFailed(["The program could not be compiled:"] + stderr.split("\n"))
        elif res.state == jagl.ExecutionResult.NOTFOUND:
            result.setFailed("gcc could not be found. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.TIMEOUT:
            result.setFailed("Timeout while calling gcc. Please inform your course instructor.")
        elif res.state == jagl.ExecutionResult.ERROR:
            result.setFailed(f'Unknown error during execution of gcc: {str(res.error)}. Please inform your course instructor.')

In [ ]:
@jagl.testcase("3", desc="Executing program", deps=["2"])
def testcase_4_3(result, suite, case):
    execresult = jagl.exec_bin("./" + executable_name, [], None, timeout = 30)
    jagl.check_execution_result(result, execresult, checkExitCode = False)
    if execresult.exitCode is not None:
        if execresult.exitCode == -11:
            result.setFailed("Program caused a segmentation fault.")
        elif execresult.exitCode < 0:
            result.setFailed(f"Program was abnormally terminated (Signal: {-execresult.exitCode})")
        elif execresult.exitCode > 0:
            result.setFailed(f"Program returned an exit code other than 0 (Exit Code: {execresult.exitCode})")